In [1]:
import pickle
import json

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

ANALISI OSPEDALI SENZA PESI

Utilizzeremo i seguenti dati: 

1. **Ospedali**


Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

In [2]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_OSPEDALI_CLEAN),
     "tipo": "Ospedali"}
     ]
Graph_functions.auto_analysis_poi(gdf_list, custom_weights={"Default": 1}, PATH_GEOJSON = "../Data/Clean/Analisi/Ospedali_nopesi.geojson")

In [3]:
gdf_analisi_ospedali_nopesi = gpd.read_file("../Data/Clean/Analisi/Ospedali_nopesi.geojson", driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [4]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_ospedali_nopesi_mod = gdf_analisi_ospedali_nopesi[gdf_analisi_ospedali_nopesi["artificial"] != True]

# Eliminiamo i percorsi completamente contenuti all'interno dei "poi" poligonari: parchi
gdf = gpd.read_file(PATH_OSPEDALI_CLEAN)
geometry_parchi = gdf.union_all()
gdf_senza_poi = []
for idx, row in gdf_analisi_ospedali_nopesi_mod.iterrows():
    if not geometry_parchi.contains(row.geometry):
        gdf_senza_poi.append(row)
gdf_analisi_ospedali_nopesi_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Salviamo per test di visualizzazione su kepler
gdf_analisi_ospedali_nopesi_mod.to_file("../Data/Clean/Analisi/ospedali_nopesi_senza_poi.geojson")

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_ospedali_nopesi_mod_most_highways = gdf_analisi_ospedali_nopesi_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [5]:
# Stampiamo il risultato
print(gdf_analisi_ospedali_nopesi_mod_most_highways)
print(f"Tot: {gdf_analisi_ospedali_nopesi_mod_most_highways["length"].sum()}")

                              length
highway                             
footway                    18.235176
tertiary                   14.042032
residential                12.269770
secondary                   8.275439
unclassified                7.910278
primary                     5.066066
cycleway                    4.725404
service                     3.901186
track                       2.363193
proposed                    0.638258
pedestrian                  0.414529
footway steps               0.407917
track path                  0.319314
path                        0.218419
cycleway pedestrian         0.174616
residential secondary       0.146541
residential footway         0.142077
footway service             0.120566
unclassified primary_link   0.087553
cycleway footway            0.087391
primary_link                0.086207
footway path                0.083698
tertiary_link               0.082140
steps                       0.061223
unclassified residential    0.046039
u